<a href="https://colab.research.google.com/github/marismendi9/sales-predictions/blob/main/sales_predictions_ParteV.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Predicción de ventas  para productos alimenticios vendidos en diversas tiendas**

## Datos personales:
* **Nombre Completo:** Mariana Arismendi
* **ID Coding Dojo:** 80511
* **Track:** Machine Learning

## **0. Introducción a los datos**

In [1]:
# Importar las librerías
import pandas as pd
import numpy as np

from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.impute import SimpleImputer
from sklearn.compose import make_column_transformer, make_column_selector
from sklearn.pipeline import make_pipeline
from sklearn.model_selection import train_test_split
from sklearn import set_config
set_config(display='diagram')

In [2]:
# Importar datos:
url = "https://raw.githubusercontent.com/marismendi9/sales-predictions/main/sales_predictions.csv"

In [3]:
#Leer archivo:
df = pd.read_csv(url)

| Nombre de la variable     | Descripción                                                                                                           |   |   |   |
|---------------------------|-----------------------------------------------------------------------------------------------------------------------|---|---|---|
| Item_Identifier           | Identificación única del producto                                                                                     |   |   |   |
| Item_Weight               | Peso del producto                                                                                                     |   |   |   |
| Item_Fat_Content          | Si el producto es bajo en grasa o regular                                                                             |   |   |   |
| Item_Visibility           | El porcentaje de la superficie total de exposición de todos los productos de una tienda asignada al producto concreto |   |   |   |
| Item_Type                 | La categoría a la que el producto pertenece                                                                           |   |   |   |
| Item_MRP                  | Precio máximo de venta al público (precio de catálogo) del producto                                                   |   |   |   |
| Outlet_Identifier         | Identificación única de la tienda                                                                                     |   |   |   |
| Outlet_Establishment_Year | El año cuando se estableció la tienda                                                                                 |   |   |   |
| Outlet_Size               | El tamaño de la tienda con respecto a la superficie                                                                   |   |   |   |
| Outlet_Location_Type      | El tipo de área donde está ubicada la tienda                                                                          |   |   |   |
| Outlet_Type               | Si el marcado es un almacén o algún tipo de supermercado                                                              |   |   |   |
| Item_Outlet_Sales         | Ventas del producto en la tienda particular Es la variable objetivo a predecir                                        |  

In [4]:
#Imprimir las primeras 5 columnas (verificar que se subió correctamente el archivo):
df.head()

,Item_Identifier,Item_Weight,Item_Fat_Content,Item_Visibility,Item_Type,Item_MRP,Outlet_Identifier,Outlet_Establishment_Year,Outlet_Size,Outlet_Location_Type,Outlet_Type,Item_Outlet_Sales
0,FDA15,9.30,Low Fat,0.016047,Dairy,249.8092,OUT049,1999,Medium,Tier 1,Supermarket Type1,3735.1380
1,DRC01,5.92,Regular,0.019278,Soft Drinks,48.2692,OUT018,2009,Medium,Tier 3,Supermarket Type2,443.4228
2,FDN15,17.50,Low Fat,0.016760,Meat,141.6180,OUT049,1999,Medium,Tier 1,Supermarket Type1,2097.2700
3,FDX07,19.20,Regular,0.000000,Fruits and Vegetables,182.0950,OUT010,1998,NaN,Tier 3,Grocery Store,732.3800
4,NCD19,8.93,Low Fat,0.000000,Household,53.8614,OUT013,1987,High,Tier 3,Supermarket Type1,994.7052


## **1. Limpieza de los datos**

In [6]:
# Chequear si hay duplicados:
df.duplicated()

# Vemos cuántos duplicados hay:
df.duplicated().sum()

0

In [7]:
# Corroboramos estén bien escritos los nombres de las columnas:
df.columns

Index(['Item_Identifier', 'Item_Weight', 'Item_Fat_Content', 'Item_Visibility',
       'Item_Type', 'Item_MRP', 'Outlet_Identifier',
       'Outlet_Establishment_Year', 'Outlet_Size', 'Outlet_Location_Type',
       'Outlet_Type', 'Item_Outlet_Sales'],
      dtype='object')

In [8]:
# Tamaño del dataframe:
df.shape

(8523, 12)

In [9]:
# Examinar el número de valores únicos de cada columna:
df.nunique()

Item_Identifier              1559
Item_Weight                   415
Item_Fat_Content                5
Item_Visibility              7880
Item_Type                      16
Item_MRP                     5938
Outlet_Identifier              10
Outlet_Establishment_Year       9
Outlet_Size                     3
Outlet_Location_Type            3
Outlet_Type                     4
Item_Outlet_Sales            3493
dtype: int64

In [10]:
# Revisar tipos de datos y valores nulos:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8523 entries, 0 to 8522
Data columns (total 12 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   Item_Identifier            8523 non-null   object 
 1   Item_Weight                7060 non-null   float64
 2   Item_Fat_Content           8523 non-null   object 
 3   Item_Visibility            8523 non-null   float64
 4   Item_Type                  8523 non-null   object 
 5   Item_MRP                   8523 non-null   float64
 6   Outlet_Identifier          8523 non-null   object 
 7   Outlet_Establishment_Year  8523 non-null   int64  
 8   Outlet_Size                6113 non-null   object 
 9   Outlet_Location_Type       8523 non-null   object 
 10  Outlet_Type                8523 non-null   object 
 11  Item_Outlet_Sales          8523 non-null   float64
dtypes: float64(4), int64(1), object(7)
memory usage: 799.2+ KB


Los tipos de datos son consistentes, procedemos a analizar los valores nulos:

In [11]:
# Identificar el número de non-values de cada columna:
df.isna().sum()

Item_Identifier                 0
Item_Weight                  1463
Item_Fat_Content                0
Item_Visibility                 0
Item_Type                       0
Item_MRP                        0
Outlet_Identifier               0
Outlet_Establishment_Year       0
Outlet_Size                  2410
Outlet_Location_Type            0
Outlet_Type                     0
Item_Outlet_Sales               0
dtype: int64

Comenzamos con Item_Weight:
Hay 8522 filas, 1559 productos diferentes y 415 diferentes pesos. Deducimos que los valores nulos están para productos que tienen su peso descrito en otras filas. O sea que por GroupBy, que omite nulos, podemos identificarlos para luego reemplazarlos.

Este proceso no crea fuga de datos ya que estamos relacionando estos valores con los valores correspondientes de otra columna, no estamos deduciendo sus valores, si no asignando los correctos teniendo en cuenta que cada producto / outlet tendrá unas características únicas.

In [12]:
# Agrupamos el Item_Identifier y el Item_Weight y luego reemplazamos los valores nulos con esta información:
x = df.groupby('Item_Identifier')['Item_Weight'].max()

df = df.set_index('Item_Identifier')
df['Item_Weight'] = df['Item_Weight'].fillna(x)
df = df.reset_index()

df.head()

,Item_Identifier,Item_Weight,Item_Fat_Content,Item_Visibility,Item_Type,Item_MRP,Outlet_Identifier,Outlet_Establishment_Year,Outlet_Size,Outlet_Location_Type,Outlet_Type,Item_Outlet_Sales
0,FDA15,9.30,Low Fat,0.016047,Dairy,249.8092,OUT049,1999,Medium,Tier 1,Supermarket Type1,3735.1380
1,DRC01,5.92,Regular,0.019278,Soft Drinks,48.2692,OUT018,2009,Medium,Tier 3,Supermarket Type2,443.4228
2,FDN15,17.50,Low Fat,0.016760,Meat,141.6180,OUT049,1999,Medium,Tier 1,Supermarket Type1,2097.2700
3,FDX07,19.20,Regular,0.000000,Fruits and Vegetables,182.0950,OUT010,1998,NaN,Tier 3,Grocery Store,732.3800
4,NCD19,8.93,Low Fat,0.000000,Household,53.8614,OUT013,1987,High,Tier 3,Supermarket Type1,994.7052


In [13]:
# Identificamos si quedan non-values en esta columna:
df.isna().sum()

Item_Identifier                 0
Item_Weight                     4
Item_Fat_Content                0
Item_Visibility                 0
Item_Type                       0
Item_MRP                        0
Outlet_Identifier               0
Outlet_Establishment_Year       0
Outlet_Size                  2410
Outlet_Location_Type            0
Outlet_Type                     0
Item_Outlet_Sales               0
dtype: int64

Siguen habiendo 4 valores nulos, por lo cual se debe tratar de ítems únicos. No vamos a rellenar estos valores hasta no dividir los datos. O podríamos eliminarlos definitivamente.

Los datos nulos restantes corresponden al tamaño de la tienda. Notamos que solo hay 10 tiendas. Procedemos a verificar si cada tienda tiene asignado su tamaño mediante un groupby para poder autocompletar los nulos con esta información

In [14]:
# Identificamos el tamaño de cada tienda
df.groupby('Outlet_Identifier')['Outlet_Size'].max()

Outlet_Identifier
OUT010       NaN
OUT013      High
OUT017       NaN
OUT018    Medium
OUT019     Small
OUT027    Medium
OUT035     Small
OUT045       NaN
OUT046     Small
OUT049    Medium
Name: Outlet_Size, dtype: object

Existen tres tiendas que no tienen la característica de tamaño definida. No vamos a llenar estos valores nulos hasta no hacer la división y a pesar de ser una categoría ordinal, no vamos a reemplazar sus nombres por valores numéricos ya que aún existen datos nulos y no se podría rellenar con un promedio (como haremos con Item_Weight que si son datos numéricos).

In [19]:
# Verificamos datos inconsistentes en las columnas categóricas:
df['Item_Fat_Content'].unique()

array(['Low Fat', 'Regular', 'low fat', 'LF', 'reg'], dtype=object)

In [21]:
# Realizamos reemplazo de variables categóricas
df['Item_Fat_Content'].replace({'Low Fat': 0, 'low fat': 0, 'LF': 0, 'reg': 1, 'Regular': 1}, inplace = True)

# Verificamos que se hayan hecho los reemplazos
df['Item_Fat_Content'].unique()

array([0, 1])

## **2. Preprocesamiento de los datos**

Si se desea saber ¿qué tan bien se pueden predecir las ventas de productos en varias basándose en el resto de características? La columna objetivo será *Item Outlet Sales*

In [22]:
# División de la validación
X = df.drop('Item_Outlet_Sales', axis=1)
y = df['Item_Outlet_Sales']
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)

In [23]:
# Instanciar selectores de columnas
cat_selector = make_column_selector(dtype_include='object')
num_selector = make_column_selector(dtype_include='number')

In [24]:
## Instanciar transformadores
# Imputers
freq_imputer = SimpleImputer(strategy='most_frequent') # Outlet_Size
mean_imputer = SimpleImputer(strategy='mean') # Item_Weight
# Scaler
scaler = StandardScaler()
# One-hot encoder
ohe = OneHotEncoder(handle_unknown='ignore', sparse=False)

In [25]:
# Pipeline numérico
numeric_pipe = make_pipeline(mean_imputer, scaler)
numeric_pipe

Pipeline(steps=[('simpleimputer', SimpleImputer()),
                ('standardscaler', StandardScaler())])

In [26]:
# Pipeline categórico
categorical_pipe = make_pipeline(freq_imputer, ohe)
categorical_pipe

Pipeline(steps=[('simpleimputer', SimpleImputer(strategy='most_frequent')),
                ('onehotencoder',
                 OneHotEncoder(handle_unknown='ignore', sparse=False))])

In [27]:
## Instanciar ColumnTransformer
# Tuplas para Column Transformer
number_tuple = (numeric_pipe, num_selector)
category_tuple = (categorical_pipe, cat_selector)

# ColumnTransformer
preprocessor = make_column_transformer(number_tuple, category_tuple)
preprocessor

ColumnTransformer(transformers=[('pipeline-1',
                                 Pipeline(steps=[('simpleimputer',
                                                  SimpleImputer()),
                                                 ('standardscaler',
                                                  StandardScaler())]),
                                 <sklearn.compose._column_transformer.make_column_selector object at 0x7f99f34a2f40>),
                                ('pipeline-2',
                                 Pipeline(steps=[('simpleimputer',
                                                  SimpleImputer(strategy='most_frequent')),
                                                 ('onehotencoder',
                                                  OneHotEncoder(handle_unknown='ignore',
                                                                sparse=False))]),
                                 <sklearn.compose._column_transformer.make_column_selector object at 0x7f99f34a2820>)])

In [28]:
## Transformador de datos
# Fit on train
preprocessor.fit(X_train)

ColumnTransformer(transformers=[('pipeline-1',
                                 Pipeline(steps=[('simpleimputer',
                                                  SimpleImputer()),
                                                 ('standardscaler',
                                                  StandardScaler())]),
                                 <sklearn.compose._column_transformer.make_column_selector object at 0x7f99f34a2f40>),
                                ('pipeline-2',
                                 Pipeline(steps=[('simpleimputer',
                                                  SimpleImputer(strategy='most_frequent')),
                                                 ('onehotencoder',
                                                  OneHotEncoder(handle_unknown='ignore',
                                                                sparse=False))]),
                                 <sklearn.compose._column_transformer.make_column_selector object at 0x7f99f34a2820>)])

In [29]:
# Transformar train y test
X_train_processed = preprocessor.transform(X_train)
X_test_processed = preprocessor.transform(X_test)

In [30]:
## Ver el resultado
# Comprobar los valores faltantes y que los datos se escalen y tengan una codificación one-hot
print(np.isnan(X_train_processed).sum().sum(), 'missing values in training data')
print(np.isnan(X_test_processed).sum().sum(), 'missing values in testing data')
print('\n')
print('All data in X_train_processed are', X_train_processed.dtype)
print('All data in X_test_processed are', X_test_processed.dtype)
print('\n')
print('shape of data is', X_train_processed.shape)
print('\n')

# Ver numpy resultante
X_train_processed

0 missing values in training data
0 missing values in testing data


All data in X_train_processed are float64
All data in X_test_processed are float64


shape of data is (6392, 1591)




array([[ 0.73647351, -0.7403206 , -0.71277507, ...,  0.        ,
         1.        ,  0.        ],
       [ 0.49910838,  1.35076614, -1.29105225, ...,  0.        ,
         1.        ,  0.        ],
       [-0.1266724 ,  1.35076614,  1.81331864, ...,  1.        ,
         0.        ,  0.        ],
       ...,
       [ 1.0062066 , -0.7403206 , -0.92052713, ...,  1.        ,
         0.        ,  0.        ],
       [ 1.59961942, -0.7403206 , -0.2277552 , ...,  1.        ,
         0.        ,  0.        ],
       [ 0.73647351, -0.7403206 , -0.95867683, ...,  1.        ,
         0.        ,  0.        ]])